In [5]:
import numpy as np
import os,sys
from PIL import Image
from helpers import *
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from pylab import *

In [103]:
# Loaded a set of images

root_dir = "augmented_training/"
#root_dir = "training/"
image_dir = root_dir + "images/"
files = os.listdir(image_dir)  

n = 300

print("Loading " + str(n) + " satellite images")
imgs = [load_image(image_dir + files[i]) for i in range(n)] #List of each image Dim (100, 400,400,3)

gt_dir = root_dir + "groundtruth/"
files2 = os.listdir(gt_dir)

print("Loading " + str(n) + " groundtruth images")
gt_imgs = [load_image(gt_dir + files2[i]) for i in range(n)]
print('Image size = ' + str(imgs[0].shape[0]) + ',' + str(imgs[0].shape[1]))

Loading 300 satellite images
Loading 300 groundtruth images
Image size = 400,400


In [104]:
patch_size = 16 # each patch is 16*16 pixels  #400/16 = 25*25 = 625 patches for 10 images

img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(n)]   # dim img_patches = (n,625,16,16,3)
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(n)]

# Linearize list of patches

img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

print("img_patches:" + str(img_patches.shape))
print("gt_patches:" + str(gt_patches.shape))

img_patches:(187500, 16, 16, 3)
gt_patches:(187500, 16, 16)


In [105]:
def build_X_Y_classes(img_patches,gt_patches,foreground_threshold):
    
    X = np.asarray([extract_features(img_patches[i]) for i in range(len(img_patches))])
    Y = np.asarray([value_to_class(np.mean(gt_patches[i]),foreground_threshold) for i in range(len(gt_patches))])

    return X,Y

In [106]:
foreground_threshold = 0.25
X,Y = build_X_Y_classes(img_patches,gt_patches,foreground_threshold)
print(X.shape)
print(Y.shape)

(187500, 6)
(187500,)


In [107]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [108]:
L_cut = int(9/10*len(X))

X_train = X[0:L_cut]
X_test = X[L_cut:len(X)]
Y_train = Y[0:L_cut]
Y_test = Y[L_cut:len(Y)]

In [117]:
model = Sequential()
model.add(Dense(64, input_dim=6, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [118]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [119]:
model.fit(X_train, Y_train,
          epochs=20,
          batch_size=128)

Epoch 1/20
168750/168750 [==============================] - 7s 44us/step - loss: 0.5373 - acc: 0.7476
Epoch 2/20
168750/168750 [==============================] - 6s 35us/step - loss: 0.5168 - acc: 0.7555
Epoch 3/20
168750/168750 [==============================] - 5s 28us/step - loss: 0.5098 - acc: 0.7621
Epoch 4/20
168750/168750 [==============================] - 4s 26us/step - loss: 0.5027 - acc: 0.7658
Epoch 5/20
168750/168750 [==============================] - 4s 26us/step - loss: 0.4916 - acc: 0.7710
Epoch 6/20
168750/168750 [==============================] - 5s 27us/step - loss: 0.4821 - acc: 0.7733
Epoch 7/20
168750/168750 [==============================] - 5s 31us/step - loss: 0.4767 - acc: 0.7756
Epoch 8/20
168750/168750 [==============================] - 5s 28us/step - loss: 0.4741 - acc: 0.7764
Epoch 9/20
168750/168750 [==============================] - 6s 37us/step - loss: 0.4715 - acc: 0.7774
Epoch 10/20
168750/168750 [==============================] - 6s 34us/step - loss: 

In [120]:
from sklearn.metrics import f1_score
score = model.evaluate(X_test, Y_test, batch_size=128)
print(score)

18750/18750 [==============================] - 1s 35us/step
[0.44863077412287394, 0.7882133334159851]


In [55]:
int(9/10*62500)


56250